In [1]:
from numpy.random import normal
import numpy as np
from numpy.linalg import inv

In [2]:
#Create the model
T = 10

#Model 1: linear model
#x_k+1 = 2*x_k + 0 + gaussian noise (mean 0,variance 0.5)
#y_k = x_k + gaussian noise  (mean 0,variance 0.5)
A = np.identity(1) * 2
B = np.zeros([1,1])
C = np.identity(1)
D = np.zeros([1,1])
Q = np.identity(1) * 5
R = np.identity(1) * 5
mean = np.zeros([1,1])

x = np.zeros([T,1])
y = np.zeros([T,1])
u = np.zeros([T,1])

print("Model 1: linear model"+ '\n' + \
"x_k+1 = "+ str(A.item()) +" * x_k + 0 + gaussian noise (mean " + str(mean.item()) +",variance "+ str(Q.item()) +")"+ '\n'+ \
"y_k = "+ str(C.item()) +" * x_k + gaussian noise  (mean "+ str(mean.item()) +",variance "+ str(R.item()) +")"+ '\n')

#Initialization
# for k in range(T-1):
#     u[k] = 0
x[0] = normal(0,1)

#Propagation
for k in range(T-1):
    x[k+1] = np.dot(A,x[k]) + np.dot(B,u[k]) + normal(mean,Q)
for k in range(T):
    y[k] = np.dot(C,x[k]) + np.dot(D,u[k]) + normal(mean,R)

print(x)
# print(y)

Model 1: linear model
x_k+1 = 2.0 * x_k + 0 + gaussian noise (mean 0.0,variance 5.0)
y_k = 1.0 * x_k + gaussian noise  (mean 0.0,variance 5.0)

[[   -1.60556601]
 [   -5.03052243]
 [  -11.06735191]
 [  -22.17098971]
 [  -42.54735874]
 [  -87.08279334]
 [ -169.79180253]
 [ -336.58796126]
 [ -668.16843124]
 [-1342.83241826]]


In [4]:
#Kalman filter
#from page 25 of ftp://icf.org.ru/pub/docs/linux-support/computer%20science/Artificial%20Intelligence/Neural%20networks/Kalman%20Filtering%20and%20Neural%20Networks%20-%20Simon%20Haykin.pdf

#Initialization
V = np.zeros([T,1])
V[0] = np.identity(1)
x_f = np.zeros([T,1])
x_f[0] = normal(0,V[0])
V_plus_all = np.zeros([T,1])
x_plus_all = np.zeros([T,1])

#Propagation
for k in range(1,T):
    x_plus = np.dot(A, x_f[k-1])
    x_plus_all[k] = x_plus
    V_plus = A.dot(V[k-1]).dot(A.T) + Q 
    V_plus_all[k] = V_plus
    
    K = V_plus.dot(C.T).dot(inv(C.dot(V_plus.dot(C.T) + R)))
    x_f[k] = x_plus + K.dot(y[k] - C.dot(x_plus))
    V[k] = (np.identity(K.shape[0]) - K.dot(C)).dot(V_plus)
    
print(x_f)
print('\n')
print(x_f - x)

[[ -4.25768755e-01]
 [ -3.71979269e+00]
 [ -1.50614902e+01]
 [ -2.41625455e+01]
 [ -3.99011826e+01]
 [ -8.07828544e+01]
 [ -1.63791640e+02]
 [ -3.34145321e+02]
 [ -6.63350417e+02]
 [ -1.33939098e+03]]


[[ 1.17979725]
 [ 1.31072974]
 [-3.99413829]
 [-1.99155575]
 [ 2.64617611]
 [ 6.29993892]
 [ 6.00016217]
 [ 2.44264003]
 [ 4.81801418]
 [ 3.44144256]]


In [5]:
#Recursive
#F = A, B = H, P = V

#Initialization
x_b = np.zeros([T,1])
x_b[-1] = x_f[-1]
V_b = np.zeros([T,1])
V_b[-1] = V[-1]

#Propagation
for k in range(T-1, 0, -1): #from T-1 to 1 by -1 increments
    print(k)
    A_b = V_plus_all[k-1].dot(A.T).dot(inv(np.matrix(V[k])))
    V_b[k-1] = V_plus_all[k-1] - A_b.dot(V[k] - V_b[k]).dot(A_b.T)
    x_b[k-1] = x_f[k-1] + A_b.dot(x_b[k] - x_plus_all[k])
    
print(x_b)   
print('\n')
print("x_f - x_b = ",'\n',x_f - x_b)
print()
print("x_f - x = ",'\n',x_f - x)
print()
print("x_b - x = ",'\n',x_b - x)

9
8
7
6
5
4
3
2
1
[[ -4.25768755e-01]
 [ -6.42570910e+08]
 [ -1.39446819e+08]
 [ -1.57133041e+07]
 [ -1.53978065e+06]
 [ -1.47672727e+05]
 [ -1.42631839e+04]
 [ -1.67406473e+03]
 [ -7.96243075e+02]
 [ -1.33939098e+03]]


x_f - x_b =  
 [[  0.00000000e+00]
 [  6.42570906e+08]
 [  1.39446804e+08]
 [  1.57132799e+07]
 [  1.53974075e+06]
 [  1.47591944e+05]
 [  1.40993922e+04]
 [  1.33991941e+03]
 [  1.32892658e+02]
 [  0.00000000e+00]]

x_f - x =  
 [[ 1.17979725]
 [ 1.31072974]
 [-3.99413829]
 [-1.99155575]
 [ 2.64617611]
 [ 6.29993892]
 [ 6.00016217]
 [ 2.44264003]
 [ 4.81801418]
 [ 3.44144256]]

x_b - x =  
 [[  1.17979725e+00]
 [ -6.42570905e+08]
 [ -1.39446808e+08]
 [ -1.57132819e+07]
 [ -1.53973811e+06]
 [ -1.47585644e+05]
 [ -1.40933921e+04]
 [ -1.33747677e+03]
 [ -1.28074644e+02]
 [  3.44144256e+00]]


We get 0, as Kalman filter does all the job on linear systems (we can use the backward recursion, but it helps only for non-linear systems)

# Now, take variable transition matrix (A_k and C_k, not A and C anymore)

In [8]:
#Create the model
T = 10

#Model 2: linear model with dependence over time
#x_k+1 = (k+1)*x_k + gaussian noise (mean 0,variance 1)
#y_k = (1 + 1/k)*x_k + gaussian noise  (mean 0,variance 0.1)
A = np.identity(1)
A_all = [A*k for k in range(1,T+1)]  #A_all is such that x[k+1] = np.dot(A_all[k],x[k]) + ...etc
B = np.zeros([1,1])
C_all = np.zeros([T,1]) #C_all is such that y[k] = np.dot(C_all[k],x[k]) + ...etc
for k in range(1,T):
    C = np.identity(1)
    C_all[k] = C*(1 + 1/(k+1))
D = np.zeros([1,1])
Q = np.identity(1) * 5
R = np.identity(1) * 5
mean = np.zeros([1,1])

x = np.zeros([T,1])
y = np.zeros([T,1])
u = np.zeros([T,1])

#Initialization
# for k in range(T-1):
#     u[k] = 0
x[0] = normal(0,1)

#Propagation
for k in range(T-1):
    x[k+1] = np.dot(A_all[k],x[k]) + np.dot(B,u[k]) + normal(mean,Q)
for k in range(T):
    y[k] = np.dot(C_all[k],x[k]) + np.dot(D,u[k]) + normal(mean,R)

print(x)
# print(y)

[[ -9.14048011e-02]
 [  4.28329091e+00]
 [  2.48625934e+00]
 [  3.23487383e+00]
 [  6.94227895e+00]
 [  4.16525026e+01]
 [  2.38758178e+02]
 [  1.67213577e+03]
 [  1.33737864e+04]
 [  1.20372856e+05]]


In [10]:
#Kalman filter
#from page 25 of ftp://icf.org.ru/pub/docs/linux-support/computer%20science/Artificial%20Intelligence/Neural%20networks/Kalman%20Filtering%20and%20Neural%20Networks%20-%20Simon%20Haykin.pdf

#Initialization
V = np.zeros([T,1])
V[0] = np.identity(1)
x_f = np.zeros([T,1])
# x_f[0] = normal(0,V[0])
x_f[0] = 1.
V_plus_all = np.zeros([T,1])
x_plus_all = np.zeros([T,1])

#Propagation
for k in range(1,T):
    x_plus = np.dot(A_all[k-1], x_f[k-1])
    x_plus_all[k] = x_plus
    V_plus = A_all[k-1].dot(V[k-1]).dot(A_all[k-1].T) + Q 
    V_plus_all[k] = V_plus
    
    K = V_plus.dot(C_all[k].T).dot(inv(C_all[k].dot(np.matrix(V_plus.dot(C_all[k].T) + R))))
    x_f[k] = x_plus + K.dot(y[k] - C_all[k].dot(x_plus))
    V[k] = (np.identity(K.shape[0]) - K.dot(C_all[k])).dot(V_plus)
    
#     print("x_plus",x_plus)
#     print(C_all[k])
#     print("h",C_all[k].dot(x_plus))
# #     print("A_all",A_all)
# #     print("C_all",C_all)
#     print("V_plus",V_plus)
#     print("K",K)
#     print("x_f[k]",x_f[k])
#     print("V[k]",V[k])
#     print()
    
print(x_f)
print('\n')
print(x_f - x)

[[  1.00000000e+00]
 [  3.37205602e+00]
 [  3.24085974e+00]
 [  6.66723782e+00]
 [  1.03311866e+01]
 [  3.69645047e+01]
 [  2.41632167e+02]
 [  1.67734818e+03]
 [  1.33800584e+04]
 [  1.20372198e+05]]


[[ 1.0914048 ]
 [-0.91123489]
 [ 0.7546004 ]
 [ 3.43236399]
 [ 3.3889077 ]
 [-4.68799784]
 [ 2.87398833]
 [ 5.21241478]
 [ 6.27198667]
 [-0.65819298]]


In [11]:
#Recursive
#F = A, B = H, P = V

#Initialization
x_b = np.zeros([T,1])
x_b[T-1] = x_f[T-1]
V_b = np.zeros([T,1])
V_b[T-1] = V[T-1]
# print(len(A_all))

#Propagation
for k in range(T-1, 0, -1): #from T-1 to 1 by -1 increments
    A_b = V_plus_all[k-1].dot(A_all[k].T).dot(inv(np.matrix(V[k])))
    V_b[k-1] = V_plus_all[k-1] - A_b.dot(V[k] - V_b[k]).dot(A_b.T)
    x_b[k-1] = x_f[k-1] + A_b.dot(x_b[k] - x_plus_all[k])
    
print(x_b)  
print('\n')
print(x_f - x_b)
print()

[[  1.00000000e+00]
 [ -2.14133745e+17]
 [ -3.49531027e+16]
 [ -2.28483128e+15]
 [ -5.67281465e+13]
 [ -6.29604028e+11]
 [ -3.72930199e+09]
 [ -1.32549247e+07]
 [ -1.71538463e+04]
 [  1.20372198e+05]]


[[  0.00000000e+00]
 [  2.14133745e+17]
 [  3.49531027e+16]
 [  2.28483128e+15]
 [  5.67281465e+13]
 [  6.29604028e+11]
 [  3.72930223e+09]
 [  1.32566020e+07]
 [  3.05339047e+04]
 [  0.00000000e+00]]

